In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def Classify_continuous(df:pd.DataFrame,col_name ):

    col=df[col_name]
    min=np.min(col)
    max=np.max(col)
    mean=np.mean([max,min])
    mean_75=np.mean([max,mean])
    mean_25=np.mean([min,mean])
    df.loc[df[col_name]<=min,col_name]=0
    df.loc[(df[col_name]>min )& (df[col_name]<=mean_25),col_name]=1
    df.loc[(df[col_name]>mean_25) & (df[col_name]<=mean),col_name]=2
    df.loc[(df[col_name]>mean) & (df[col_name]<=mean_75),col_name]=3
    df.loc[(df[col_name]>mean_75) & (df[col_name]<=max),col_name]=4
    df.loc[df[col_name]>max ,col_name]=5


In [ ]:
def clean_data(df:pd.DataFrame):


    # drop the non coreleation columns
    df=df.drop(columns=["hot","is_guest_login","logged_in","is_host_login","num_outbound_cmds","target","num_failed_logins","land","num_root","wrong_fragment","su_attempted","num_file_creations","is_guest_login","urgent","is_host_login","num_outbound_cmds","num_access_files","num_shells"])

    df.loc[(df["duration"]<=0),"duration"]=0
    df.loc[(df["duration"]>0),"duration"]=1
    map_protocol_type={
        "icmp":0,
        "tcp":1,
        "udp":2
    }
    df["protocol_type"]=df["protocol_type"].map(map_protocol_type)

    map_service = {"ecr_i": 0, "smtp": 1, "http": 2, "private": 3,"domain_u":4}
    df["service"] = df["service"].map(map_service).fillna(5).astype(int)

    map_flag={"SF":0,"S0":1,"REJ":2,"RSTO":4,"RSTR":3}
    df["flag"]=df["flag"].map(map_flag).fillna(5).astype(int)
    col_continuos=["src_bytes","diff_srv_rate","dst_host_same_src_port_rate"]
    for s in col_continuos:
        Classify_continuous(df,col_name=s)

    df.loc[(df["dst_bytes"]==0),"dst_bytes"]=0
    df.loc[(df["dst_bytes"]!=0),"dst_bytes"]=1

    df.loc[(df["num_compromised"]==0),"num_compromised"]=0
    df.loc[(df["num_compromised"]!=0),"num_compromised"]=1

    df.loc[(df["count"]>20),"count"]=1
    df.loc[(df["count"]<=20),"count"]=0

    df.loc[(df["srv_count"]>20),"srv_count"]=1
    df.loc[(df["srv_count"]<=20),"srv_count"]=0


    df.loc[(df["serror_rate"]==0),"serror_rate"]=0
    df.loc[(df["serror_rate"]!=0),"serror_rate"]=1

    df.loc[(df["srv_serror_rate"]==0),"srv_serror_rate"]=0
    df.loc[(df["srv_serror_rate"]!=0),"srv_serror_rate"]=1

    df.loc[(df["rerror_rate"]==0),"rerror_rate"]=0
    df.loc[(df["rerror_rate"]!=0),"rerror_rate"]=1

    df.loc[(df["srv_rerror_rate"]==0),"srv_rerror_rate"]=0
    df.loc[(df["srv_rerror_rate"]!=0),"srv_rerror_rate"]=1

    df.loc[(df["same_srv_rate"]==1),"same_srv_rate"]=0
    df.loc[(df["same_srv_rate"]!=1),"same_srv_rate"]=1

    df.loc[(df["srv_diff_host_rate"]==0),"srv_diff_host_rate"]=0
    df.loc[(df["srv_diff_host_rate"]!=0),"srv_diff_host_rate"]=1

    df.loc[(df["dst_host_count"]==255),"dst_host_count"]=0
    df.loc[(df["dst_host_count"]!=255),"dst_host_count"]=1

    df.loc[(df["dst_host_srv_count"]==255),"dst_host_srv_count"]=0
    df.loc[(df["dst_host_srv_count"]!=255),"dst_host_srv_count"]=1


    df.loc[(df["dst_host_same_srv_rate"]==1),"dst_host_same_srv_rate"]=0
    df.loc[(df["dst_host_same_srv_rate"]!=1),"dst_host_same_srv_rate"]=1

    df.loc[(df["dst_host_diff_srv_rate"]==0),"dst_host_diff_srv_rate"]=0
    df.loc[(df["dst_host_diff_srv_rate"]!=0),"dst_host_diff_srv_rate"]=1

    df.loc[(df["dst_host_srv_diff_host_rate"]==0),"dst_host_srv_diff_host_rate"]=0
    df.loc[(df["dst_host_srv_diff_host_rate"]!=0),"dst_host_srv_diff_host_rate"]=1

    df.loc[(df["dst_host_srv_rerror_rate"]==0),"dst_host_srv_rerror_rate"]=0
    df.loc[(df["dst_host_srv_rerror_rate"]!=0),"dst_host_srv_rerror_rate"]=1

    df.loc[(df["dst_host_rerror_rate"]==0),"dst_host_rerror_rate"]=0
    df.loc[(df["dst_host_rerror_rate"]!=0),"dst_host_rerror_rate"]=1

    df.loc[(df["dst_host_srv_serror_rate"]==0),"dst_host_srv_serror_rate"]=0
    df.loc[(df["dst_host_srv_serror_rate"]!=0),"dst_host_srv_serror_rate"]=1

    df.loc[(df["dst_host_serror_rate"]==0),"dst_host_serror_rate"]=0
    df.loc[(df["dst_host_serror_rate"]!=0),"dst_host_serror_rate"]=1


    return df





In [ ]:
def count_values_columns(df:pd.DataFrame,cols:list):
    for s in cols:
        print(df[s].value_counts())


In [ ]:
def split_X_y(df,y_label="class"):
    y=df[y_label]
    X=df.drop(columns=[y_label])
    return X,y

In [ ]:
def split_train_test_valid(df, random_state=42, test_size=0.2, valid_size=0.2):
    n = len(df)
    np.random.seed(random_state)
    indexes = np.random.permutation(n)


    test_size_df = int(n * test_size)
    valid_size_df = int((n - test_size_df) * valid_size)
    train_size_df = n - (test_size_df + valid_size_df)


    df_train = df.iloc[indexes[:train_size_df]].reset_index(drop=True)
    df_test = df.iloc[indexes[train_size_df:train_size_df + test_size_df]].reset_index(drop=True)
    df_valid = df.iloc[indexes[train_size_df + test_size_df:]].reset_index(drop=True)

    return df_train, df_test, df_valid

In [ ]:
def gini_impurity(y):
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / counts.sum()
    return 1 - np.sum(probabilities ** 2)

def weighted_gini(X_feature, y):

    unique_values = np.unique(X_feature)
    gini_values = []

    for value in unique_values:
        left_split = y[X_feature == value]
        right_split = y[X_feature != value]

        gini_left = gini_impurity(left_split)
        gini_right = gini_impurity(right_split)


        weight_left = len(left_split) / len(y)
        weight_right = len(right_split) / len(y)

        weighted_gini = (weight_left * gini_left) + (weight_right * gini_right)
        gini_values.append((value, weighted_gini))

    return min(gini_values, key=lambda x: x[1])

def best_feature_to_split(X, y):
    best_feature = None
    best_gini = float("inf")
    best_threshold = None

    for feature in X.columns:
        threshold, gini = weighted_gini(X[feature], y)

        if gini < best_gini:
            best_gini = gini
            best_feature = feature
            best_threshold = threshold

    return best_feature, best_threshold, best_gini

In [ ]:
class Node:

    def __init__(self, feature=None, children=None, gini=None, entropy=None, value=None):
        self.feature = feature
        self.children = children or {}
        self.gini = gini
        self.entropy = entropy
        self.value = value

class DecisionTree:
    def __init__(self, criterion="gini", max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def fit(self, X, y):
        full_data = pd.DataFrame(X).copy()
        full_data["label"] = y
        self.root = self._build_tree(full_data, depth=0)

    def _build_tree(self, data, depth):

        y = data["label"]
        root_gini = self._gini(y)
        root_entropy = self._entropy(y)
        root_label=self._compute_label(y)
        if (self.max_depth is not None and depth >= self.max_depth) or \
        (len(data) < self.min_samples_split) or (len(y) == 0) or \
        (len(y.unique()) == 1):
            return Node(value=root_label, gini=root_gini, entropy=root_entropy)


        best_feature = self._best_split_feature(data)

        if best_feature is None:
            return Node(value=root_label, gini=root_gini, entropy=root_entropy)

        node = Node(feature=best_feature, gini=root_gini, entropy=root_entropy, children={})

        for category in data[best_feature].unique():
            subset = data[data[best_feature] == category].drop(columns=[best_feature])

            if len(subset) < self.min_samples_leaf:
                node.children[category] = Node(value=root_label, gini=root_gini, entropy=root_entropy)
            else:
                node.children[category] = self._build_tree(subset, depth + 1)

        return node

    def _best_split_feature(self, data):

        best_feature = None
        best_score = 1 if self.criterion == "gini" else -1

        for feature in data.columns[:-1]:
            score = self._split_score(data, feature)

            if (self.criterion == "gini" and score < best_score) or \
               (self.criterion == "entropy" and score > best_score):
                best_feature, best_score = feature, score

        return best_feature
    def _compute_label(self, y):

        y_unique, count = np.unique(y, return_counts=True)
        most_common_value = y_unique[np.argmax(count)]
        return most_common_value
        pass


    def _split_score(self, data, feature):
        total = len(data)
        target = data["label"]
        weighted_impurity = 0

        for category in data[feature].unique():
            subset = data[data[feature] == category]["label"]
            weight = len(subset) / total
            impurity = self._gini(subset) if self.criterion == "gini" else self._entropy(subset)
            weighted_impurity += weight * impurity

        return weighted_impurity

    def _gini(self, y):
        proportions = y.value_counts(normalize=True)
        return 1 - np.sum(proportions ** 2)

    def _entropy(self, y):
        proportions = y.value_counts(normalize=True)
        return -np.sum(proportions * np.log2(proportions + 1e-9))

    def predict(self, X):
        return [self._predict_one(row, self.root) for _, row in pd.DataFrame(X).iterrows()]

    def _predict_one(self, row, node):

        if node.value is not None:
            return node.value

        feature_value = row.get(node.feature)

        if feature_value in node.children:
            return self._predict_one(row, node.children[feature_value])
        else:
            return self._compute_label_from_node(node)
    def _compute_label_from_node(self, node):

        values = [child.value for child in node.children.values() if child.value is not None]
        return max(set(values), key=values.count) if values else None



In [ ]:
def accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

In [ ]:
def change_indexes(i, j, k, t, max_i, max_j, max_k, max_t):
    if t + 1 < max_t:
        return i, j, k, t + 1

    t = 0
    if k + 1 < max_k:
        return i, j, k + 1, t

    k = 0
    if j + 1 < max_j:
        return i, j + 1, k, t

    j = 0
    if i + 1 < max_i:
        return i + 1, j, k, t

    return i, j, k, t


In [ ]:
def hyperparameter_tuning(dict_values, X_train, y_train, X_valid, y_valid):
    result = {}
    max_valid = 0
    best_model = None
    i, j, k, t = 0, 0, 0, 0

    max_i = len(dict_values["max_depth"])
    max_j = len(dict_values["min_samples_leaf"])
    max_k = len(dict_values["min_samples_split"])
    max_t = len(dict_values["criterion"])

    for _ in range(max_i * max_j * max_k * max_t):
        max_depth = dict_values["max_depth"][i]
        min_samples_leaf = dict_values["min_samples_leaf"][j]
        min_samples_split = dict_values["min_samples_split"][k]
        criterion = dict_values["criterion"][t]

        model = DecisionTree(max_depth=max_depth, min_samples_leaf=min_samples_leaf,
                             min_samples_split=min_samples_split, criterion=criterion)
        model.fit(X_train, y_train)
        y_pred_valid = model.predict(X_valid)
        y_pred_train = model.predict(X_train)

        train_acc = accuracy(y_train, y_pred_train)
        valid_acc = accuracy(y_valid, y_pred_valid)

        result[f"max_depth:{max_depth}, min_samples_leaf:{min_samples_leaf}, "
               f"min_samples_split:{min_samples_split}, criterion:{criterion}"] = f"train:{train_acc}, valid:{valid_acc}"

        i, j, k, t = change_indexes(i, j, k, t, max_i, max_j, max_k, max_t)

    return result


## start of the project

In [ ]:
df=pd.read_csv("Airplane.csv")
df=clean_data(df)

count_values_columns(df,df.columns.to_list())


In [ ]:
train_df,test_df,valid_df=split_train_test_valid(df,random_state=42,test_size=0.2,valid_size=0.2)
x_train,y_train=split_X_y(train_df,y_label="Attack Type")
x_test,y_test=split_X_y(test_df,y_label="Attack Type")
x_valid,y_valid=split_X_y(valid_df,y_label="Attack Type")
print(best_feature_to_split(x_train,y_train))

('dst_bytes', np.int64(0), np.float64(0.08431970393670667))


In [ ]:
hyperparameter={
    "max_depth":[9,10],
    "min_samples_leaf":[20,25],
    "min_samples_split":[20,30,35],
    "criterion":["gini","entropy"]
}
res=hyperparameter_tuning(hyperparameter,x_train,y_train,x_valid,y_valid)
res

In [ ]:
#baced on the the res I choose the max_depth=9 and min_samples_leaf=20 and min_sample_split=30 and gini as the criterion
best_model=DecisionTree(max_depth=9,min_samples_leaf=20,min_samples_split=30,criterion="gini")
best_model.fit(x_train,y_train)
y_pred=best_model.predict(x_test)
y_pred_train=best_model.predict(x_train)
print(f"test: {accuracy(y_pred,y_test)}")
print(f"train: {accuracy(y_train,y_pred_train)}")

0.9968827172988948
0.9969636972047038
